In [ ]:
# bmw_topic_cards.py  –  v6  (2025‑05‑02)
# ================================================================
# • Stratified random sampling (mirrors topic sentiment mix)
# • One paragraph summary  +  “‑ ” negative bullets  +  “+ ” positive bullets
# • Prompts stored inside TopicSummary.prompts  (audit / debugging)
# • No JSON parsing, so Ollama chatter can’t break the pipeline
# ---------------------------------------------------------------

from __future__ import annotations

import html
import logging
import subprocess
import textwrap
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass, field
from typing import Callable, Dict, List, Optional

import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
import pandas as pd
from IPython.display import HTML, display
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm

# -------------------------------------------------------------------
#  CONFIGURATION
# -------------------------------------------------------------------
TOP_N_TOPICS        = 15          # how many topics to visualise
PROMPT_SAMPLE_SIZE  = 300         # reviews fed to each LLM prompt
N_KEYWORDS          = 8
N_PHRASES           = 5
MAX_BULLETS         = 7           # issues / positives
BMW_BLUE            = "#0066B1"

STAR_SVG = (
    "<svg width='14' height='14' viewBox='0 0 24 24' "
    "xmlns='http://www.w3.org/2000/svg' style='vertical-align:-2px'>"
    "<polygon fill='#FFD700' "
    "points='12 2 15.09 8.26 22 9.27 17 14.14 "
    "18.18 21.02 12 17.77 5.82 21.02 7 14.14 2 9.27 8.91 8.26'/></svg>"
)

# -------------------------------------------------------------------
#  NLTK – bootstrap stop‑words
# -------------------------------------------------------------------
for res in ("stopwords", "punkt"):
    try:
        nltk.data.find(f"corpora/{res}")
    except LookupError:
        nltk.download(res, quiet=True)

STOPWORDS = set(stopwords.words("english")) | {
    "bmw", "app", "car", "please", "would", "also", "get", "use", "using"
}

# -------------------------------------------------------------------
#  OLLAMA RUNNER  (temperature 0 for determinism)
# -------------------------------------------------------------------
def _subprocess_runner(prompt: str, model: str) -> str:
    proc = subprocess.run(
        ["ollama", "run", model, "-t", "0"],
        input=prompt,
        text=True,
        capture_output=True,
        check=False,
    )
    return proc.stdout

# -------------------------------------------------------------------
#  STRATIFIED SAMPLING  (mirror sentiment mix)
# -------------------------------------------------------------------
def _stratified_sample(df: pd.DataFrame, size: int, seed: int = 0) -> List[str]:
    dist = df["sentiment"].value_counts(normalize=True)
    quota = {s: int(round(dist.get(s, 0) * size)) for s in ("negative", "positive", "neutral")}
    diff = size - sum(quota.values())
    if diff:
        quota[max(dist, key=dist.get, default="negative")] += diff

    col = "content_english" if "content_english" in df.columns else "content"
    texts: List[str] = []

    for sentiment, n in quota.items():
        if n <= 0:
            continue
        bucket = df[df["sentiment"] == sentiment]
        sample_n = min(n, len(bucket))
        texts.extend(bucket.sample(sample_n, random_state=seed)[col].tolist())

    # top‑up if any bucket ran short
    if len(texts) < size:
        short = size - len(texts)
        remainder = df.drop(df.index[df[col].isin(texts)]).sample(short, random_state=seed)
        texts.extend(remainder[col].tolist())

    return texts

# -------------------------------------------------------------------
#  BULLET‑PARSER
# -------------------------------------------------------------------
def _parse_bullets(text: str, prefix: str) -> List[str]:
    """Return list of lines starting with the prefix (‘- ’ or ‘+ ’)."""
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    bullets = [ln[len(prefix):].strip() for ln in lines if ln.startswith(prefix)]
    # fallback to comma‑sep if model ignored prefixes
    if not bullets and "," in text:
        bullets = [part.strip() for part in text.split(",") if part.strip()]
    return bullets[:MAX_BULLETS]

# -------------------------------------------------------------------
#  DATACLASS
# -------------------------------------------------------------------
@dataclass
class TopicSummary:
    summary: str
    issues: List[str]
    positives: List[str]
    avg_rating: float
    review_count: int
    sentiment_dist: Dict[str, float]
    top_keywords: List[str]
    top_phrases: List[str]
    prompts: Dict[str, str] = field(default_factory=dict)

    @property
    def stars(self) -> str:
        full, half = int(self.avg_rating), self.avg_rating - int(self.avg_rating) >= 0.5
        return STAR_SVG * full + (STAR_SVG if half else "")

# -------------------------------------------------------------------
#  MAIN ENTRY
# -------------------------------------------------------------------
def create_topic_cards_from_classified(
    df: pd.DataFrame,
    *,
    ollama_model_name: str = "gemma3:12b",
    ollama_runner: Callable[[str, str], str] = _subprocess_runner,
) -> Dict[str, TopicSummary]:
    """
    Render topic cards & a rating chart; return dict[topic] -> TopicSummary.
    Prompts are stored in `TopicSummary.prompts`.
    """
    log = _setup_logger()

    # -------- sanity checks -----------------------------------------
    if {"topics", "sentiment"} - set(df.columns):
        raise ValueError("DataFrame must include 'topics' & 'sentiment'.")

    text_col = "content_english" if "content_english" in df.columns else "content"

    if "score" not in df.columns:
        df["score"] = (
            df["sentiment"]
            .map({"positive": 4.5, "neutral": 3.0, "negative": 1.5})
            .fillna(3.0)
        )

    # -------- pick top topics --------------------------------------
    topics_freq = df["topics"].str.get_dummies(sep=",").sum()
    top_topics = (
        topics_freq.sort_values(ascending=False).head(TOP_N_TOPICS).index
    )

    results: Dict[str, TopicSummary] = {}

    for topic in tqdm(top_topics, desc="Topics"):
        sub = df[df["topics"].str.contains(fr"\b{topic}\b", case=False, na=False)]
        if sub.empty:
            continue

        avg = sub["score"].mean()
        mix = sub["sentiment"].value_counts(normalize=True).to_dict()

        tokens = word_tokenize(" ".join(sub[text_col].fillna("").str.lower()))
        tokens = [t for t in tokens if t.isalpha() and t not in STOPWORDS and len(t) > 2]
        keywords = [w for w, _ in Counter(tokens).most_common(N_KEYWORDS)]
        bigrams = [f"{a} {b}" for a, b in zip(tokens, tokens[1:])]
        phrases = [p for p, _ in Counter(bigrams).most_common(N_PHRASES)]

        sample_n = min(PROMPT_SAMPLE_SIZE, len(sub))
        sample_texts = _stratified_sample(sub, sample_n)
        sample_blob = "\n".join(f"[{i}] {txt}" for i, txt in enumerate(sample_texts))

        ctx_block = (
            f"Average score {avg:.2f} (n={len(sub)}). "
            f"Sentiment mix: neg {mix.get('negative',0):.0%}, "
            f"pos {mix.get('positive',0):.0%}, "
            f"neu {mix.get('neutral',0):.0%}.\n"
            f"Top keywords: {', '.join(keywords)}.\n"
            f"Top phrases: {', '.join(phrases)}."
        )

        prompts: Dict[str, str] = {}

        def build(kind: str) -> str:
            """Return fully‑formed prompt & store it"""
            header = (
                "You are an analytical assistant. Follow ALL rules:\n"
                f"• Focus **only** on the topic '{topic}'.\n"
                "• Ignore unrelated features.\n"
                "• Use English even if reviews were translated.\n"
                "• Be factual, concise (temperature 0).\n"
            )
            if kind == "summary":
                body = (
                    "Write one concise paragraph (3‑5 sentences) that "
                    "summarises what users say about this topic, covering "
                    "both pain points and praise."
                )
            else:
                label = "negative issues" if kind == "issues" else "positive aspects"
                prefix = "-" if kind == "issues" else "+"
                body = (
                    f"List up to {MAX_BULLETS} main {label}. "
                    f"Each line MUST start with '{prefix} '. "
                    "Use short noun phrases, no duplication, no periods."
                )

            prompt = (
                f"{header}\n\nCONTEXT:\n{ctx_block}\n\nSAMPLES:\n{sample_blob}\n\nTASK:\n{body}"
            )
            prompts[kind] = prompt
            return prompt

        # ----- call Ollama (three prompts in parallel) ----------------
        with ThreadPoolExecutor(max_workers=3) as pool:
            futures = {
                pool.submit(ollama_runner, build(k), ollama_model_name): k
                for k in ("summary", "issues", "positives")
            }
            raw = {k: f.result().strip() for f, k in futures.items()}

        summary_text = raw["summary"]
        issues_list  = _parse_bullets(raw["issues"], "-")
        pos_list     = _parse_bullets(raw["positives"], "+")

        results[topic] = TopicSummary(
            summary=summary_text,
            issues=issues_list,
            positives=pos_list,
            avg_rating=avg,
            review_count=len(sub),
            sentiment_dist=mix,
            top_keywords=keywords,
            top_phrases=phrases,
            prompts=prompts,
        )

    # ------------- render & plot -------------------------------------
    _render_cards(results)
    _plot_chart(results, df)
    return {k: v.__dict__ for k, v in results.items()}

# -------------------------------------------------------------------
#  VISUAL HELPER – HTML CARDS
# -------------------------------------------------------------------
def _render_cards(data: Dict[str, TopicSummary]) -> None:
    css = textwrap.dedent(
        """
        <style>
          :root {{--blue:{blue};--neg:#E53935;--pos:#43A047;--neu:#FFA000;
                 --bg:#fff;--text:#1a1a1a;}}
          @media (prefers-color-scheme: dark) {{
              :root {{--bg:#121212;--text:#e0e0e0;}}
          }}
          body {{background:var(--bg);color:var(--text);}}

/* container */
          .cards {{max-width:1200px;margin:0 auto;font-family:'Helvetica Neue',Arial,sans-serif;}}

/* card */
          .card {{
              display:grid;grid-template-columns:220px 1fr;
              border:1px solid #bbb;border-radius:8px;margin:18px 0;
              overflow:hidden;box-shadow:0 4px 10px rgba(0,0,0,.12);
              animation:fadeIn .4s ease;
          }}
          @keyframes fadeIn {{from {{opacity:0;transform:translateY(8px);}}
                              to   {{opacity:1;transform:translateY(0);}}}}
          header {{
              grid-column:1/-1;background:linear-gradient(135deg,var(--blue) 0%,#0088cc 100%);
              color:#fff;padding:12px 18px;font-size:20px;font-weight:600;
          }}

/* sidebar */
          .side {{
              background:#f5f7fa;border-right:1px solid #ddd;
              padding:14px;display:flex;flex-direction:column;gap:14px;
          }}
          .stat .num {{font-weight:700;font-size:19px;line-height:1;color:var(--text);}}
          .stat .label{{font-size:11px;text-transform:uppercase;font-weight:600;
                       letter-spacing:.4px;color:var(--text);}}
          .bar {{display:flex;height:10px;border-radius:4px;overflow:hidden;margin-top:4px;}}
          .seg {{flex-shrink:0;transition:opacity .2s;}} .seg:hover {{opacity:.75;}}
          .neg{{background:var(--neg);}} .pos{{background:var(--pos);}} .neu{{background:var(--neu);}}

/* main */
          .main {{padding:18px;line-height:1.6;}}
          .summary {{
              background:rgba(0,102,177,.07);border-left:3px solid var(--blue);
              padding:10px;margin-bottom:16px;line-height:1.6;
          }}
          .cols {{display:flex;flex-wrap:wrap;gap:24px;margin-bottom:16px;}}
          h4 {{margin:0 0 6px;color:var(--blue);font-weight:600;}}
          ul {{margin:0;padding-left:18px;}}
          li {{margin:4px 0;}}

/* chips */
          .chips {{margin-bottom:10px;}}
          .chip {{
              display:inline-block;padding:4px 10px;border-radius:20px;
              font-size:11px;font-weight:700;margin:2px;cursor:pointer;
              transition:opacity .2s;
          }}
          .chip:hover {{opacity:.8;}}
          .kw{{background:#e3f2fd;color:var(--blue);}}
          .ph{{background:#e8f5e9;color:#2e7d32;border:1px solid #c8e6c9;}}

/* responsive */
          @media (max-width:768px){{
              .card{{grid-template-columns:1fr}}
              .side{{order:2;border:none;border-top:1px solid #ddd;
                    flex-direction:row;justify-content:space-around}}
          }}
        </style>
        """
    ).format(blue=BMW_BLUE)

    output: List[str] = [css, "<div class='cards'>"]

    # sort by review count
    for topic, info in sorted(data.items(), key=lambda x: x[1].review_count, reverse=True):
        if topic.lower() == "other":
            continue

        bar = "".join(_bar_seg(name, frac) for name, frac in info.sentiment_dist.items())
        kw_html = "".join(f"<span class='chip kw'>{html.escape(k)}</span>" for k in info.top_keywords)
        ph_html = "".join(f"<span class='chip ph'>{html.escape(p)}</span>" for p in info.top_phrases)
        issues_html = "".join(f"<li>{html.escape(i)}</li>" for i in info.issues) or "<li>No major issues</li>"
        pos_html = "".join(f"<li>{html.escape(p)}</li>" for p in info.positives) or "<li>No specific positives</li>"

        output.append(
            f"<div class='card'>"
            f"<header>{html.escape(topic)}</header>"

            f"<section class='side'>"
            f"<div class='stat'><span class='num'>{info.review_count}</span><span class='label'>Reviews</span></div>"
            f"<div class='stat'><span class='num'>{info.avg_rating:.2f}/5 {info.stars}</span>"
            f"<span class='label'>Average</span></div>"
            f"<div class='stat'><span class='label'>Sentiment</span><div class='bar'>{bar}</div></div>"
            "</section>"

            "<section class='main'>"
            f"<p class='summary'>{html.escape(info.summary)}</p>"
            "<div class='cols'>"
            f"<div><h4>Issues</h4><ul>{issues_html}</ul></div>"
            f"<div><h4>Positives</h4><ul>{pos_html}</ul></div>"
            "</div>"
            f"<div class='chips'><h4>Keywords</h4>{kw_html}</div>"
            f"<div class='chips'><h4>Phrases</h4>{ph_html}</div>"
            "</section>"
            "</div>"
        )

    output.append("</div>")
    display(HTML("\n".join(output)))

def _bar_seg(name: str, frac: float) -> str:
    cls = "pos" if name == "positive" else "neg" if name == "negative" else "neu"
    pct = max(frac * 100, 1)
    return f"<span class='seg {cls}' style='width:{pct}%' title='{name}: {frac:.1%}'></span>"

# -------------------------------------------------------------------
#  VISUAL HELPER – MATPLOTLIB CHART
# -------------------------------------------------------------------
def _plot_chart(data: Dict[str, TopicSummary], df: pd.DataFrame) -> None:
    rows = [(t, d.avg_rating, d.review_count) for t, d in data.items() if t.lower() != "other"]
    if not rows:
        return
    topics, ratings, counts = zip(*rows)
    order = sorted(range(len(topics)), key=lambda i: ratings[i])
    topics = [topics[i] for i in order]
    ratings = [ratings[i] for i in order]
    counts = [counts[i] for i in order]

    cmap = mpl.colormaps.get_cmap("viridis").resampled(len(topics))

    plt.figure(figsize=(12, 10))
    bars = plt.barh(topics, ratings, color=[cmap(i) for i in range(len(topics))])

    for bar, cnt, rating in zip(bars, counts, ratings):
        x = rating + 0.05 if rating < 2.5 else rating - 0.6
        clr = "black" if rating < 2.5 else "white"
        plt.text(x, bar.get_y() + bar.get_height() / 2, f"n={cnt}", va="center",
                 color=clr, fontweight="bold")

    mean = df["score"].mean()
    plt.axvline(mean, linestyle="--", linewidth=2, label=f"Overall {mean:.2f}")
    plt.xticks(range(1, 6), ["★" * i for i in range(1, 6)])
    plt.xlim(0, 5.2)
    plt.xlabel("Star Rating")
    plt.ylabel("Topic")
    plt.title("BMW App – Average Rating by Topic")
    plt.legend()
    plt.grid(axis="x", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

# -------------------------------------------------------------------
#  MISC HELPERS
# -------------------------------------------------------------------
def _setup_logger() -> logging.Logger:
    logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
    return logging.getLogger("bmw_topic_cards")

In [ ]:
# bmw_topic_cards.py  –  v6  (2025‑05‑02)
# ================================================================
# • Stratified random sampling (mirrors topic sentiment mix)
# • One paragraph summary  +  “‑ ” negative bullets  +  “+ ” positive bullets
# • Prompts stored inside TopicSummary.prompts  (audit / debugging)
# • No JSON parsing, so Ollama chatter can’t break the pipeline
# ---------------------------------------------------------------

from __future__ import annotations

import html
import logging
import subprocess
import textwrap
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass, field
from typing import Callable, Dict, List, Optional

import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
import pandas as pd
from IPython.display import HTML, display
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm

# -------------------------------------------------------------------
#  CONFIGURATION
# -------------------------------------------------------------------
TOP_N_TOPICS        = 15          # how many topics to visualise
PROMPT_SAMPLE_SIZE  = 300         # reviews fed to each LLM prompt
N_KEYWORDS          = 8
N_PHRASES           = 5
MAX_BULLETS         = 7           # issues / positives
BMW_BLUE            = "#0066B1"

STAR_SVG = (
    "<svg width='14' height='14' viewBox='0 0 24 24' "
    "xmlns='http://www.w3.org/2000/svg' style='vertical-align:-2px'>"
    "<polygon fill='#FFD700' "
    "points='12 2 15.09 8.26 22 9.27 17 14.14 "
    "18.18 21.02 12 17.77 5.82 21.02 7 14.14 2 9.27 8.91 8.26'/></svg>"
)

# -------------------------------------------------------------------
#  NLTK – bootstrap stop‑words
# -------------------------------------------------------------------
for res in ("stopwords", "punkt"):
    try:
        nltk.data.find(f"corpora/{res}")
    except LookupError:
        nltk.download(res, quiet=True)

STOPWORDS = set(stopwords.words("english")) | {
    "bmw", "app", "car", "please", "would", "also", "get", "use", "using"
}

# -------------------------------------------------------------------
#  OLLAMA RUNNER  (temperature 0 for determinism)
# -------------------------------------------------------------------
def _subprocess_runner(prompt: str, model: str) -> str:
    proc = subprocess.run(
        ["ollama", "run", model, "-t", "0"],
        input=prompt,
        text=True,
        capture_output=True,
        check=False,
    )
    return proc.stdout

# -------------------------------------------------------------------
#  STRATIFIED SAMPLING  (mirror sentiment mix)
# -------------------------------------------------------------------
def _stratified_sample(df: pd.DataFrame, size: int, seed: int = 0) -> List[str]:
    dist = df["sentiment"].value_counts(normalize=True)
    quota = {s: int(round(dist.get(s, 0) * size)) for s in ("negative", "positive", "neutral")}
    diff = size - sum(quota.values())
    if diff:
        quota[max(dist, key=dist.get, default="negative")] += diff

    col = "content_english" if "content_english" in df.columns else "content"
    texts: List[str] = []

    for sentiment, n in quota.items():
        if n <= 0:
            continue
        bucket = df[df["sentiment"] == sentiment]
        sample_n = min(n, len(bucket))
        texts.extend(bucket.sample(sample_n, random_state=seed)[col].tolist())

    # top‑up if any bucket ran short
    if len(texts) < size:
        short = size - len(texts)
        remainder = df.drop(df.index[df[col].isin(texts)]).sample(short, random_state=seed)
        texts.extend(remainder[col].tolist())

    return texts

# -------------------------------------------------------------------
#  BULLET‑PARSER
# -------------------------------------------------------------------
def _parse_bullets(text: str, prefix: str) -> List[str]:
    """Return list of lines starting with the prefix (‘- ’ or ‘+ ’)."""
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    bullets = [ln[len(prefix):].strip() for ln in lines if ln.startswith(prefix)]
    # fallback to comma‑sep if model ignored prefixes
    if not bullets and "," in text:
        bullets = [part.strip() for part in text.split(",") if part.strip()]
    return bullets[:MAX_BULLETS]

# -------------------------------------------------------------------
#  DATACLASS
# -------------------------------------------------------------------
@dataclass
class TopicSummary:
    summary: str
    issues: List[str]
    positives: List[str]
    avg_rating: float
    review_count: int
    sentiment_dist: Dict[str, float]
    top_keywords: List[str]
    top_phrases: List[str]
    prompts: Dict[str, str] = field(default_factory=dict)

    @property
    def stars(self) -> str:
        full, half = int(self.avg_rating), self.avg_rating - int(self.avg_rating) >= 0.5
        return STAR_SVG * full + (STAR_SVG if half else "")

# -------------------------------------------------------------------
#  MAIN ENTRY
# -------------------------------------------------------------------
def create_topic_cards_from_classified(
    df: pd.DataFrame,
    *,
    ollama_model_name: str = "gemma3:12b",
    ollama_runner: Callable[[str, str], str] = _subprocess_runner,
) -> Dict[str, TopicSummary]:
    """
    Render topic cards & a rating chart; return dict[topic] -> TopicSummary.
    Prompts are stored in `TopicSummary.prompts`.
    """
    log = _setup_logger()

    # -------- sanity checks -----------------------------------------
    if {"topics", "sentiment"} - set(df.columns):
        raise ValueError("DataFrame must include 'topics' & 'sentiment'.")

    text_col = "content_english" if "content_english" in df.columns else "content"

    if "score" not in df.columns:
        df["score"] = (
            df["sentiment"]
            .map({"positive": 4.5, "neutral": 3.0, "negative": 1.5})
            .fillna(3.0)
        )

    # -------- pick top topics --------------------------------------
    topics_freq = df["topics"].str.get_dummies(sep=",").sum()
    top_topics = (
        topics_freq.sort_values(ascending=False).head(TOP_N_TOPICS).index
    )

    results: Dict[str, TopicSummary] = {}

    for topic in tqdm(top_topics, desc="Topics"):
        sub = df[df["topics"].str.contains(fr"\b{topic}\b", case=False, na=False)]
        if sub.empty:
            continue

        avg = sub["score"].mean()
        mix = sub["sentiment"].value_counts(normalize=True).to_dict()

        tokens = word_tokenize(" ".join(sub[text_col].fillna("").str.lower()))
        tokens = [t for t in tokens if t.isalpha() and t not in STOPWORDS and len(t) > 2]
        keywords = [w for w, _ in Counter(tokens).most_common(N_KEYWORDS)]
        bigrams = [f"{a} {b}" for a, b in zip(tokens, tokens[1:])]
        phrases = [p for p, _ in Counter(bigrams).most_common(N_PHRASES)]

        sample_n = min(PROMPT_SAMPLE_SIZE, len(sub))
        sample_texts = _stratified_sample(sub, sample_n)
        sample_blob = "\n".join(f"[{i}] {txt}" for i, txt in enumerate(sample_texts))

        ctx_block = (
            f"Average score {avg:.2f} (n={len(sub)}). "
            f"Sentiment mix: neg {mix.get('negative',0):.0%}, "
            f"pos {mix.get('positive',0):.0%}, "
            f"neu {mix.get('neutral',0):.0%}.\n"
            f"Top keywords: {', '.join(keywords)}.\n"
            f"Top phrases: {', '.join(phrases)}."
        )

        prompts: Dict[str, str] = {}

        def build(kind: str) -> str:
            """Return fully‑formed prompt & store it"""
            header = (
                "You are an analytical assistant. Follow ALL rules:\n"
                f"• Focus **only** on the topic '{topic}'.\n"
                "• Ignore unrelated features.\n"
                "• Use English even if reviews were translated.\n"
                "• Be factual, concise (temperature 0).\n"
            )
            if kind == "summary":
                body = (
                    "Write one concise paragraph (3‑5 sentences) that "
                    "summarises what users say about this topic, covering "
                    "both pain points and praise."
                )
            else:
                label = "negative issues" if kind == "issues" else "positive aspects"
                prefix = "-" if kind == "issues" else "+"
                body = (
                    f"List up to {MAX_BULLETS} main {label}. "
                    f"Each line MUST start with '{prefix} '. "
                    "Use short noun phrases, no duplication, no periods."
                )

            prompt = (
                f"{header}\n\nCONTEXT:\n{ctx_block}\n\nSAMPLES:\n{sample_blob}\n\nTASK:\n{body}"
            )
            prompts[kind] = prompt
            return prompt

        # ----- call Ollama (three prompts in parallel) ----------------
        with ThreadPoolExecutor(max_workers=3) as pool:
            futures = {
                pool.submit(ollama_runner, build(k), ollama_model_name): k
                for k in ("summary", "issues", "positives")
            }
            raw = {k: f.result().strip() for f, k in futures.items()}

        summary_text = raw["summary"]
        issues_list  = _parse_bullets(raw["issues"], "-")
        pos_list     = _parse_bullets(raw["positives"], "+")

        results[topic] = TopicSummary(
            summary=summary_text,
            issues=issues_list,
            positives=pos_list,
            avg_rating=avg,
            review_count=len(sub),
            sentiment_dist=mix,
            top_keywords=keywords,
            top_phrases=phrases,
            prompts=prompts,
        )

    # ------------- render & plot -------------------------------------
    _render_cards(results)
    _plot_chart(results, df)
    return {k: v.__dict__ for k, v in results.items()}

# -------------------------------------------------------------------
#  VISUAL HELPER – HTML CARDS
# -------------------------------------------------------------------
def _render_cards(data: Dict[str, TopicSummary]) -> None:
    css = textwrap.dedent(
        """
        <style>
          :root {{--blue:{blue};--neg:#E53935;--pos:#43A047;--neu:#FFA000;
                 --bg:#fff;--text:#1a1a1a;}}
          @media (prefers-color-scheme: dark) {{
              :root {{--bg:#121212;--text:#e0e0e0;}}
          }}
          body {{background:var(--bg);color:var(--text);}}

/* container */
          .cards {{max-width:1200px;margin:0 auto;font-family:'Helvetica Neue',Arial,sans-serif;}}

/* card */
          .card {{
              display:grid;grid-template-columns:220px 1fr;
              border:1px solid #bbb;border-radius:8px;margin:18px 0;
              overflow:hidden;box-shadow:0 4px 10px rgba(0,0,0,.12);
              animation:fadeIn .4s ease;
          }}
          @keyframes fadeIn {{from {{opacity:0;transform:translateY(8px);}}
                              to   {{opacity:1;transform:translateY(0);}}}}
          header {{
              grid-column:1/-1;background:linear-gradient(135deg,var(--blue) 0%,#0088cc 100%);
              color:#fff;padding:12px 18px;font-size:20px;font-weight:600;
          }}

/* sidebar */
          .side {{
              background:#f5f7fa;border-right:1px solid #ddd;
              padding:14px;display:flex;flex-direction:column;gap:14px;
          }}
          .stat .num {{font-weight:700;font-size:19px;line-height:1;color:var(--text);}}
          .stat .label{{font-size:11px;text-transform:uppercase;font-weight:600;
                       letter-spacing:.4px;color:var(--text);}}
          .bar {{display:flex;height:10px;border-radius:4px;overflow:hidden;margin-top:4px;}}
          .seg {{flex-shrink:0;transition:opacity .2s;}} .seg:hover {{opacity:.75;}}
          .neg{{background:var(--neg);}} .pos{{background:var(--pos);}} .neu{{background:var(--neu);}}

/* main */
          .main {{padding:18px;line-height:1.6;}}
          .summary {{
              background:rgba(0,102,177,.07);border-left:3px solid var(--blue);
              padding:10px;margin-bottom:16px;line-height:1.6;
          }}
          .cols {{display:flex;flex-wrap:wrap;gap:24px;margin-bottom:16px;}}
          h4 {{margin:0 0 6px;color:var(--blue);font-weight:600;}}
          ul {{margin:0;padding-left:18px;}}
          li {{margin:4px 0;}}

/* chips */
          .chips {{margin-bottom:10px;}}
          .chip {{
              display:inline-block;padding:4px 10px;border-radius:20px;
              font-size:11px;font-weight:700;margin:2px;cursor:pointer;
              transition:opacity .2s;
          }}
          .chip:hover {{opacity:.8;}}
          .kw{{background:#e3f2fd;color:var(--blue);}}
          .ph{{background:#e8f5e9;color:#2e7d32;border:1px solid #c8e6c9;}}

/* responsive */
          @media (max-width:768px){{
              .card{{grid-template-columns:1fr}}
              .side{{order:2;border:none;border-top:1px solid #ddd;
                    flex-direction:row;justify-content:space-around}}
          }}
        </style>
        """
    ).format(blue=BMW_BLUE)

    output: List[str] = [css, "<div class='cards'>"]

    # sort by review count
    for topic, info in sorted(data.items(), key=lambda x: x[1].review_count, reverse=True):
        if topic.lower() == "other":
            continue

        bar = "".join(_bar_seg(name, frac) for name, frac in info.sentiment_dist.items())
        kw_html = "".join(f"<span class='chip kw'>{html.escape(k)}</span>" for k in info.top_keywords)
        ph_html = "".join(f"<span class='chip ph'>{html.escape(p)}</span>" for p in info.top_phrases)
        issues_html = "".join(f"<li>{html.escape(i)}</li>" for i in info.issues) or "<li>No major issues</li>"
        pos_html = "".join(f"<li>{html.escape(p)}</li>" for p in info.positives) or "<li>No specific positives</li>"

        output.append(
            f"<div class='card'>"
            f"<header>{html.escape(topic)}</header>"

            f"<section class='side'>"
            f"<div class='stat'><span class='num'>{info.review_count}</span><span class='label'>Reviews</span></div>"
            f"<div class='stat'><span class='num'>{info.avg_rating:.2f}/5 {info.stars}</span>"
            f"<span class='label'>Average</span></div>"
            f"<div class='stat'><span class='label'>Sentiment</span><div class='bar'>{bar}</div></div>"
            "</section>"

            "<section class='main'>"
            f"<p class='summary'>{html.escape(info.summary)}</p>"
            "<div class='cols'>"
            f"<div><h4>Issues</h4><ul>{issues_html}</ul></div>"
            f"<div><h4>Positives</h4><ul>{pos_html}</ul></div>"
            "</div>"
            f"<div class='chips'><h4>Keywords</h4>{kw_html}</div>"
            f"<div class='chips'><h4>Phrases</h4>{ph_html}</div>"
            "</section>"
            "</div>"
        )

    output.append("</div>")
    display(HTML("\n".join(output)))

def _bar_seg(name: str, frac: float) -> str:
    cls = "pos" if name == "positive" else "neg" if name == "negative" else "neu"
    pct = max(frac * 100, 1)
    return f"<span class='seg {cls}' style='width:{pct}%' title='{name}: {frac:.1%}'></span>"

# -------------------------------------------------------------------
#  VISUAL HELPER – MATPLOTLIB CHART
# -------------------------------------------------------------------
def _plot_chart(data: Dict[str, TopicSummary], df: pd.DataFrame) -> None:
    rows = [(t, d.avg_rating, d.review_count) for t, d in data.items() if t.lower() != "other"]
    if not rows:
        return
    topics, ratings, counts = zip(*rows)
    order = sorted(range(len(topics)), key=lambda i: ratings[i])
    topics = [topics[i] for i in order]
    ratings = [ratings[i] for i in order]
    counts = [counts[i] for i in order]

    cmap = mpl.colormaps.get_cmap("viridis").resampled(len(topics))

    plt.figure(figsize=(12, 10))
    bars = plt.barh(topics, ratings, color=[cmap(i) for i in range(len(topics))])

    for bar, cnt, rating in zip(bars, counts, ratings):
        x = rating + 0.05 if rating < 2.5 else rating - 0.6
        clr = "black" if rating < 2.5 else "white"
        plt.text(x, bar.get_y() + bar.get_height() / 2, f"n={cnt}", va="center",
                 color=clr, fontweight="bold")

    mean = df["score"].mean()
    plt.axvline(mean, linestyle="--", linewidth=2, label=f"Overall {mean:.2f}")
    plt.xticks(range(1, 6), ["★" * i for i in range(1, 6)])
    plt.xlim(0, 5.2)
    plt.xlabel("Star Rating")
    plt.ylabel("Topic")
    plt.title("BMW App – Average Rating by Topic")
    plt.legend()
    plt.grid(axis="x", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

# -------------------------------------------------------------------
#  MISC HELPERS
# -------------------------------------------------------------------
def _setup_logger() -> logging.Logger:
    logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
    return logging.getLogger("bmw_topic_cards")

In [ ]:
# bmw_topic_cards.py  –  v6  (2025‑05‑02)
# ================================================================
# • Stratified random sampling (mirrors topic sentiment mix)
# • One paragraph summary  +  “‑ ” negative bullets  +  “+ ” positive bullets
# • Prompts stored inside TopicSummary.prompts  (audit / debugging)
# • No JSON parsing, so Ollama chatter can’t break the pipeline
# ---------------------------------------------------------------

from __future__ import annotations

import html
import logging
import subprocess
import textwrap
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass, field
from typing import Callable, Dict, List, Optional

import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
import pandas as pd
from IPython.display import HTML, display
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm

# -------------------------------------------------------------------
#  CONFIGURATION
# -------------------------------------------------------------------
TOP_N_TOPICS        = 15          # how many topics to visualise
PROMPT_SAMPLE_SIZE  = 300         # reviews fed to each LLM prompt
N_KEYWORDS          = 8
N_PHRASES           = 5
MAX_BULLETS         = 7           # issues / positives
BMW_BLUE            = "#0066B1"

STAR_SVG = (
    "<svg width='14' height='14' viewBox='0 0 24 24' "
    "xmlns='http://www.w3.org/2000/svg' style='vertical-align:-2px'>"
    "<polygon fill='#FFD700' "
    "points='12 2 15.09 8.26 22 9.27 17 14.14 "
    "18.18 21.02 12 17.77 5.82 21.02 7 14.14 2 9.27 8.91 8.26'/></svg>"
)

# -------------------------------------------------------------------
#  NLTK – bootstrap stop‑words
# -------------------------------------------------------------------
for res in ("stopwords", "punkt"):
    try:
        nltk.data.find(f"corpora/{res}")
    except LookupError:
        nltk.download(res, quiet=True)

STOPWORDS = set(stopwords.words("english")) | {
    "bmw", "app", "car", "please", "would", "also", "get", "use", "using"
}

# -------------------------------------------------------------------
#  OLLAMA RUNNER  (temperature 0 for determinism)
# -------------------------------------------------------------------
def _subprocess_runner(prompt: str, model: str) -> str:
    proc = subprocess.run(
        ["ollama", "run", model, "-t", "0"],
        input=prompt,
        text=True,
        capture_output=True,
        check=False,
    )
    return proc.stdout

# -------------------------------------------------------------------
#  STRATIFIED SAMPLING  (mirror sentiment mix)
# -------------------------------------------------------------------
def _stratified_sample(df: pd.DataFrame, size: int, seed: int = 0) -> List[str]:
    dist = df["sentiment"].value_counts(normalize=True)
    quota = {s: int(round(dist.get(s, 0) * size)) for s in ("negative", "positive", "neutral")}
    diff = size - sum(quota.values())
    if diff:
        quota[max(dist, key=dist.get, default="negative")] += diff

    col = "content_english" if "content_english" in df.columns else "content"
    texts: List[str] = []

    for sentiment, n in quota.items():
        if n <= 0:
            continue
        bucket = df[df["sentiment"] == sentiment]
        sample_n = min(n, len(bucket))
        texts.extend(bucket.sample(sample_n, random_state=seed)[col].tolist())

    # top‑up if any bucket ran short
    if len(texts) < size:
        short = size - len(texts)
        remainder = df.drop(df.index[df[col].isin(texts)]).sample(short, random_state=seed)
        texts.extend(remainder[col].tolist())

    return texts

# -------------------------------------------------------------------
#  BULLET‑PARSER
# -------------------------------------------------------------------
def _parse_bullets(text: str, prefix: str) -> List[str]:
    """Return list of lines starting with the prefix (‘- ’ or ‘+ ’)."""
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    bullets = [ln[len(prefix):].strip() for ln in lines if ln.startswith(prefix)]
    # fallback to comma‑sep if model ignored prefixes
    if not bullets and "," in text:
        bullets = [part.strip() for part in text.split(",") if part.strip()]
    return bullets[:MAX_BULLETS]

# -------------------------------------------------------------------
#  DATACLASS
# -------------------------------------------------------------------
@dataclass
class TopicSummary:
    summary: str
    issues: List[str]
    positives: List[str]
    avg_rating: float
    review_count: int
    sentiment_dist: Dict[str, float]
    top_keywords: List[str]
    top_phrases: List[str]
    prompts: Dict[str, str] = field(default_factory=dict)

    @property
    def stars(self) -> str:
        full, half = int(self.avg_rating), self.avg_rating - int(self.avg_rating) >= 0.5
        return STAR_SVG * full + (STAR_SVG if half else "")

# -------------------------------------------------------------------
#  MAIN ENTRY
# -------------------------------------------------------------------
def create_topic_cards_from_classified(
    df: pd.DataFrame,
    *,
    ollama_model_name: str = "gemma3:12b",
    ollama_runner: Callable[[str, str], str] = _subprocess_runner,
) -> Dict[str, TopicSummary]:
    """
    Render topic cards & a rating chart; return dict[topic] -> TopicSummary.
    Prompts are stored in `TopicSummary.prompts`.
    """
    log = _setup_logger()

    # -------- sanity checks -----------------------------------------
    if {"topics", "sentiment"} - set(df.columns):
        raise ValueError("DataFrame must include 'topics' & 'sentiment'.")

    text_col = "content_english" if "content_english" in df.columns else "content"

    if "score" not in df.columns:
        df["score"] = (
            df["sentiment"]
            .map({"positive": 4.5, "neutral": 3.0, "negative": 1.5})
            .fillna(3.0)
        )

    # -------- pick top topics --------------------------------------
    topics_freq = df["topics"].str.get_dummies(sep=",").sum()
    top_topics = (
        topics_freq.sort_values(ascending=False).head(TOP_N_TOPICS).index
    )

    results: Dict[str, TopicSummary] = {}

    for topic in tqdm(top_topics, desc="Topics"):
        sub = df[df["topics"].str.contains(fr"\b{topic}\b", case=False, na=False)]
        if sub.empty:
            continue

        avg = sub["score"].mean()
        mix = sub["sentiment"].value_counts(normalize=True).to_dict()

        tokens = word_tokenize(" ".join(sub[text_col].fillna("").str.lower()))
        tokens = [t for t in tokens if t.isalpha() and t not in STOPWORDS and len(t) > 2]
        keywords = [w for w, _ in Counter(tokens).most_common(N_KEYWORDS)]
        bigrams = [f"{a} {b}" for a, b in zip(tokens, tokens[1:])]
        phrases = [p for p, _ in Counter(bigrams).most_common(N_PHRASES)]

        sample_n = min(PROMPT_SAMPLE_SIZE, len(sub))
        sample_texts = _stratified_sample(sub, sample_n)
        sample_blob = "\n".join(f"[{i}] {txt}" for i, txt in enumerate(sample_texts))

        ctx_block = (
            f"Average score {avg:.2f} (n={len(sub)}). "
            f"Sentiment mix: neg {mix.get('negative',0):.0%}, "
            f"pos {mix.get('positive',0):.0%}, "
            f"neu {mix.get('neutral',0):.0%}.\n"
            f"Top keywords: {', '.join(keywords)}.\n"
            f"Top phrases: {', '.join(phrases)}."
        )

        prompts: Dict[str, str] = {}

        def build(kind: str) -> str:
            """Return fully‑formed prompt & store it"""
            header = (
                "You are an analytical assistant. Follow ALL rules:\n"
                f"• Focus **only** on the topic '{topic}'.\n"
                "• Ignore unrelated features.\n"
                "• Use English even if reviews were translated.\n"
                "• Be factual, concise (temperature 0).\n"
            )
            if kind == "summary":
                body = (
                    "Write one concise paragraph (3‑5 sentences) that "
                    "summarises what users say about this topic, covering "
                    "both pain points and praise."
                )
            else:
                label = "negative issues" if kind == "issues" else "positive aspects"
                prefix = "-" if kind == "issues" else "+"
                body = (
                    f"List up to {MAX_BULLETS} main {label}. "
                    f"Each line MUST start with '{prefix} '. "
                    "Use short noun phrases, no duplication, no periods."
                )

            prompt = (
                f"{header}\n\nCONTEXT:\n{ctx_block}\n\nSAMPLES:\n{sample_blob}\n\nTASK:\n{body}"
            )
            prompts[kind] = prompt
            return prompt

        # ----- call Ollama (three prompts in parallel) ----------------
        with ThreadPoolExecutor(max_workers=3) as pool:
            futures = {
                pool.submit(ollama_runner, build(k), ollama_model_name): k
                for k in ("summary", "issues", "positives")
            }
            raw = {k: f.result().strip() for f, k in futures.items()}

        summary_text = raw["summary"]
        issues_list  = _parse_bullets(raw["issues"], "-")
        pos_list     = _parse_bullets(raw["positives"], "+")

        results[topic] = TopicSummary(
            summary=summary_text,
            issues=issues_list,
            positives=pos_list,
            avg_rating=avg,
            review_count=len(sub),
            sentiment_dist=mix,
            top_keywords=keywords,
            top_phrases=phrases,
            prompts=prompts,
        )

    # ------------- render & plot -------------------------------------
    _render_cards(results)
    _plot_chart(results, df)
    return {k: v.__dict__ for k, v in results.items()}

# -------------------------------------------------------------------
#  VISUAL HELPER – HTML CARDS
# -------------------------------------------------------------------
def _render_cards(data: Dict[str, TopicSummary]) -> None:
    css = textwrap.dedent(
        """
        <style>
          :root {{--blue:{blue};--neg:#E53935;--pos:#43A047;--neu:#FFA000;
                 --bg:#fff;--text:#1a1a1a;}}
          @media (prefers-color-scheme: dark) {{
              :root {{--bg:#121212;--text:#e0e0e0;}}
          }}
          body {{background:var(--bg);color:var(--text);}}

/* container */
          .cards {{max-width:1200px;margin:0 auto;font-family:'Helvetica Neue',Arial,sans-serif;}}

/* card */
          .card {{
              display:grid;grid-template-columns:220px 1fr;
              border:1px solid #bbb;border-radius:8px;margin:18px 0;
              overflow:hidden;box-shadow:0 4px 10px rgba(0,0,0,.12);
              animation:fadeIn .4s ease;
          }}
          @keyframes fadeIn {{from {{opacity:0;transform:translateY(8px);}}
                              to   {{opacity:1;transform:translateY(0);}}}}
          header {{
              grid-column:1/-1;background:linear-gradient(135deg,var(--blue) 0%,#0088cc 100%);
              color:#fff;padding:12px 18px;font-size:20px;font-weight:600;
          }}

/* sidebar */
          .side {{
              background:#f5f7fa;border-right:1px solid #ddd;
              padding:14px;display:flex;flex-direction:column;gap:14px;
          }}
          .stat .num {{font-weight:700;font-size:19px;line-height:1;color:var(--text);}}
          .stat .label{{font-size:11px;text-transform:uppercase;font-weight:600;
                       letter-spacing:.4px;color:var(--text);}}
          .bar {{display:flex;height:10px;border-radius:4px;overflow:hidden;margin-top:4px;}}
          .seg {{flex-shrink:0;transition:opacity .2s;}} .seg:hover {{opacity:.75;}}
          .neg{{background:var(--neg);}} .pos{{background:var(--pos);}} .neu{{background:var(--neu);}}

/* main */
          .main {{padding:18px;line-height:1.6;}}
          .summary {{
              background:rgba(0,102,177,.07);border-left:3px solid var(--blue);
              padding:10px;margin-bottom:16px;line-height:1.6;
          }}
          .cols {{display:flex;flex-wrap:wrap;gap:24px;margin-bottom:16px;}}
          h4 {{margin:0 0 6px;color:var(--blue);font-weight:600;}}
          ul {{margin:0;padding-left:18px;}}
          li {{margin:4px 0;}}

/* chips */
          .chips {{margin-bottom:10px;}}
          .chip {{
              display:inline-block;padding:4px 10px;border-radius:20px;
              font-size:11px;font-weight:700;margin:2px;cursor:pointer;
              transition:opacity .2s;
          }}
          .chip:hover {{opacity:.8;}}
          .kw{{background:#e3f2fd;color:var(--blue);}}
          .ph{{background:#e8f5e9;color:#2e7d32;border:1px solid #c8e6c9;}}

/* responsive */
          @media (max-width:768px){{
              .card{{grid-template-columns:1fr}}
              .side{{order:2;border:none;border-top:1px solid #ddd;
                    flex-direction:row;justify-content:space-around}}
          }}
        </style>
        """
    ).format(blue=BMW_BLUE)

    output: List[str] = [css, "<div class='cards'>"]

    # sort by review count
    for topic, info in sorted(data.items(), key=lambda x: x[1].review_count, reverse=True):
        if topic.lower() == "other":
            continue

        bar = "".join(_bar_seg(name, frac) for name, frac in info.sentiment_dist.items())
        kw_html = "".join(f"<span class='chip kw'>{html.escape(k)}</span>" for k in info.top_keywords)
        ph_html = "".join(f"<span class='chip ph'>{html.escape(p)}</span>" for p in info.top_phrases)
        issues_html = "".join(f"<li>{html.escape(i)}</li>" for i in info.issues) or "<li>No major issues</li>"
        pos_html = "".join(f"<li>{html.escape(p)}</li>" for p in info.positives) or "<li>No specific positives</li>"

        output.append(
            f"<div class='card'>"
            f"<header>{html.escape(topic)}</header>"

            f"<section class='side'>"
            f"<div class='stat'><span class='num'>{info.review_count}</span><span class='label'>Reviews</span></div>"
            f"<div class='stat'><span class='num'>{info.avg_rating:.2f}/5 {info.stars}</span>"
            f"<span class='label'>Average</span></div>"
            f"<div class='stat'><span class='label'>Sentiment</span><div class='bar'>{bar}</div></div>"
            "</section>"

            "<section class='main'>"
            f"<p class='summary'>{html.escape(info.summary)}</p>"
            "<div class='cols'>"
            f"<div><h4>Issues</h4><ul>{issues_html}</ul></div>"
            f"<div><h4>Positives</h4><ul>{pos_html}</ul></div>"
            "</div>"
            f"<div class='chips'><h4>Keywords</h4>{kw_html}</div>"
            f"<div class='chips'><h4>Phrases</h4>{ph_html}</div>"
            "</section>"
            "</div>"
        )

    output.append("</div>")
    display(HTML("\n".join(output)))

def _bar_seg(name: str, frac: float) -> str:
    cls = "pos" if name == "positive" else "neg" if name == "negative" else "neu"
    pct = max(frac * 100, 1)
    return f"<span class='seg {cls}' style='width:{pct}%' title='{name}: {frac:.1%}'></span>"

# -------------------------------------------------------------------
#  VISUAL HELPER – MATPLOTLIB CHART
# -------------------------------------------------------------------
def _plot_chart(data: Dict[str, TopicSummary], df: pd.DataFrame) -> None:
    rows = [(t, d.avg_rating, d.review_count) for t, d in data.items() if t.lower() != "other"]
    if not rows:
        return
    topics, ratings, counts = zip(*rows)
    order = sorted(range(len(topics)), key=lambda i: ratings[i])
    topics = [topics[i] for i in order]
    ratings = [ratings[i] for i in order]
    counts = [counts[i] for i in order]

    cmap = mpl.colormaps.get_cmap("viridis").resampled(len(topics))

    plt.figure(figsize=(12, 10))
    bars = plt.barh(topics, ratings, color=[cmap(i) for i in range(len(topics))])

    for bar, cnt, rating in zip(bars, counts, ratings):
        x = rating + 0.05 if rating < 2.5 else rating - 0.6
        clr = "black" if rating < 2.5 else "white"
        plt.text(x, bar.get_y() + bar.get_height() / 2, f"n={cnt}", va="center",
                 color=clr, fontweight="bold")

    mean = df["score"].mean()
    plt.axvline(mean, linestyle="--", linewidth=2, label=f"Overall {mean:.2f}")
    plt.xticks(range(1, 6), ["★" * i for i in range(1, 6)])
    plt.xlim(0, 5.2)
    plt.xlabel("Star Rating")
    plt.ylabel("Topic")
    plt.title("BMW App – Average Rating by Topic")
    plt.legend()
    plt.grid(axis="x", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

# -------------------------------------------------------------------
#  MISC HELPERS
# -------------------------------------------------------------------
def _setup_logger() -> logging.Logger:
    logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
    return logging.getLogger("bmw_topic_cards")